In [362]:
import json
import os
import sys
from pprint import pprint
import concurrent.futures
import pandas as pd
import numpy as np
from Bio import SeqIO
from intervaltree import Interval, IntervalTree
import seaborn as sns
import plotnine as p9

In [328]:
def reducer(old, new):
    return "%s;%s" % (old, new)

def get_location(location):
    loc_ = location.split(":")
    return int(loc_[0].lstrip("[")), int(loc_[1].rstrip("]"))

def get_antismash_and_knowncluster(json_file):
    try:
        with open(json_file) as fp:
            try:
                bgc_list = []
                bin_id = os.path.basename(json_file).rpartition(".")[0]
                mgs_type = json_file.strip().split("/")[1]
                json_dict = json.load(fp)
                record_num = 0
                for i in json_dict["records"]:
                    interval_list = []
                    record_num += 1
                    
                    if "antismash.detection.hmm_detection" in i["modules"]:       
                        for bgc in i["modules"]["antismash.detection.hmm_detection"]["rule_results"]["cds_by_protocluster"]:
                            bgc_begin, bgc_end = get_location(bgc[0]["location"])
                            protocluster_type = bgc[0]["qualifiers"]["product"][0]
                            bgc_tuple = (bgc_begin, bgc_end, protocluster_type)
                            interval_list.append(bgc_tuple)

                    if "antismash.detection.clusterfinder_probabilistic" in i["modules"]:
                        for bgc in i["modules"]["antismash.detection.clusterfinder_probabilistic"]["areas"]:
                            bgc_begin, bgc_end = get_location(bgc[0])
                            interval_list.append((bgc_begin, bgc_end, "unknown"))
                    
                    bgc_tree = IntervalTree(Interval(begin, end, protocluster_type) for begin, end, protocluster_type in interval_list)
                    bgc_tree.merge_overlaps(data_reducer=reducer)

                    region_num = 0          
                    if "antismash.modules.clusterblast" in i["modules"]:
                        
                        if len(i["modules"]["antismash.modules.clusterblast"]["knowncluster"]["results"]) != len(bgc_tree):
                            print("error: %s %s %d: total region number not equal knowncluster results number: %d != %d" % \
                                  (bin_id, i["id"], record_num,
                                   len(bgc_tree),
                                   len(i["modules"]["antismash.modules.clusterblast"]["knowncluster"]["results"])))
                        
                        for kc, bi in zip(i["modules"]["antismash.modules.clusterblast"]["knowncluster"]["results"], \
                                          sorted(bgc_tree)):
                            region_num += 1
                            
                            kc_total_hits = kc["total_hits"]
                            kc_acc = ""
                            kc_desc = ""
                            kc_type = ""
                            kc_hits = 0
                            kc_core_gene_hits = 0
                            kc_blast_score = 0
                            kc_synteny_score = 0
                            kc_core_bonus = 0
                            if kc_total_hits > 0:
                                kc_acc = kc["ranking"][0][0]["accession"]
                                kc_desc = kc["ranking"][0][0]["description"]
                                kc_type = kc["ranking"][0][0]["cluster_type"]
                                kc_hits = kc["ranking"][0][1]["hits"]
                                kc_core_gene_hits = kc["ranking"][0][1]["core_gene_hits"]
                                kc_blast_score = kc["ranking"][0][1]["blast_score"]
                                kc_synteny_score = kc["ranking"][0][1]["synteny_score"]
                                kc_core_bonus = kc["ranking"][0][1]["core_bonus"]
                            
                            #protocluster_type = ""
                            bgc_dict = {
                                "bin_id": bin_id,
                                "mgs_type": mgs_type,
                                "record_id": i["id"],
                                "record_region": "region %d.%d" % (record_num, region_num),
                                "protocluster_type": bi.data,
                                "bgc_location": "%d:%d" % (bi.begin+1, bi.end),
                                "kc_most_similar": kc_desc,
                                "kc_known_cluster": kc_type,
                                "kc_bgc_acc": kc_acc,
                                "kc_total_hits": kc_total_hits,
                                "kc_hits": kc_hits,
                                "kc_core_gene_hits": kc_core_gene_hits,
                                "kc_blast_score": kc_blast_score,
                                "kc_synteny_score": kc_synteny_score,
                                "kc_core_bonus": kc_core_bonus
                            }
                            #pprint(bgc_dict)
                            bgc_list.append(bgc_dict)
                return pd.DataFrame(bgc_list)
            except json.JSONDecodeError:
                print("Json file %s does not exist" % json_file)
                return None
    except:
        print("error occurred while parsing json:", sys.exc_info()[1])

In [325]:
def get_antismash_protocluster(json_file):
    try:
        with open(json_file) as fp:
            try:
                bin_id = os.path.basename(json_file).rpartition(".")[0]
                mgs_type = json_file.strip().split("/")[1]
                df = pd.DataFrame(columns=["bin_id", "mgs_type", "record_id",
                                           "bgc_location", "bgc_core_location",
                                           "detection_rule", "protocluster_type", "gene_list"])
                json_dict = json.load(fp)
                for i in json_dict["records"]:
                    if "antismash.detection.hmm_detection" in i["modules"]:
                        protocluster = i["modules"]["antismash.detection.hmm_detection"]["rule_results"]["cds_by_protocluster"]
                        if len(protocluster) > 0:
                            count = 0
                            for bgc in protocluster:
                                gene_list = []
                                for gene in bgc[1]:
                                    gene_list.append(gene["cds_name"])
                                gene_list_str = ";".join(gene_list)
                                
                                df = df.append({
                                    "bin_id": bin_id,
                                    "mgs_type": mgs_type,
                                    "record_id": i["id"],
                                    "bgc_location": bgc[0]["location"],
                                    "bgc_core_location": ";".join(bgc[0]["qualifiers"]["core_location"]),
                                    "detection_rule": ";".join(bgc[0]["qualifiers"]["detection_rule"]),
                                    "protocluster_type": ";".join(bgc[0]["qualifiers"]["product"]),
                                    "gene_list": gene_list_str}, ignore_index=True)
      
                    if "antismash.detection.clusterfinder_probabilistic" in i["modules"]:
                        areas = i["modules"]["antismash.detection.clusterfinder_probabilistic"]["areas"]
                        if len(areas) > 0:
                            for bgc in areas:
                                df = df.append({
                                    "bin_id": bin_id,
                                    "mgs_type": mgs_type,
                                    "record_id": i["id"],
                                    "bgc_location": bgc[0],
                                    "bgc_core_location": "",
                                    "detection_rule": "",
                                    "protocluster_type": "unknown",
                                    "gene_list": ""}, ignore_index=True)
                return df
            except json.JSONDecodeError:
                print("Json file %s does not exist" % json_file)
                return None
    except:
        print("error occurred while parsing json:", sys.exc_info()[1])
        return None

In [321]:
def get_all_antismash(json_files, func, workers):
    df_list = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
        for df in executor.map(func, json_files):
            if not df is None:
                df_list.append(df)
    
    df_ = pd.concat(df_list)
    return df_

In [326]:
a = get_antismash_protocluster("results/kmgs/20298_3_36/20298_3_36.json")
a

,bin_id,mgs_type,record_id,bgc_location,bgc_core_location,detection_rule,protocluster_type,gene_list
0,20298_3_36,kmgs,c00001_gnlBGI..,[486700:497176],,,unknown,
1,20298_3_36,kmgs,c00002_gnlBGI..,[396799:416946],[406799:406946](+),(subtilosin or thuricin or TIGR04404 or TIGR03...,sactipeptide,LEJECIEP_01079;LEJECIEP_01080;LEJECIEP_01085
2,20298_3_36,kmgs,c00003_gnlBGI..,[43482:52668],,,unknown,
3,20298_3_36,kmgs,c00003_gnlBGI..,[162778:173998],,,unknown,
4,20298_3_36,kmgs,c00004_gnlBGI..,[38143:91874],[58143:71874],(cds(Condensation and (AMP-binding or A-OX)) o...,NRPS,LEJECIEP_01659;LEJECIEP_01670;LEJECIEP_01672;L...
5,20298_3_36,kmgs,c00004_gnlBGI..,[106400:129154],[116400:119154](-),((LANC_like and (Lant_dehydr_N or Lant_dehydr_...,lanthipeptide,LEJECIEP_01709
6,20298_3_36,kmgs,c00004_gnlBGI..,[43477:73308],,,unknown,


In [329]:
b = get_antismash_and_knowncluster("results/kmgs/20298_3_36/20298_3_36.json")
b

,bin_id,mgs_type,record_id,record_region,protocluster_type,bgc_location,kc_most_similar,kc_known_cluster,kc_bgc_acc,kc_total_hits,kc_hits,kc_core_gene_hits,kc_blast_score,kc_synteny_score,kc_core_bonus
0,20298_3_36,kmgs,c00001_gnlBGI..,region 1.1,unknown,486701:497176,,,,0,0,0,0.0,0,0
1,20298_3_36,kmgs,c00002_gnlBGI..,region 2.1,sactipeptide,396800:416946,,,,0,0,0,0.0,0,0
2,20298_3_36,kmgs,c00003_gnlBGI..,region 3.1,unknown,43483:52668,,,,0,0,0,0.0,0,0
3,20298_3_36,kmgs,c00003_gnlBGI..,region 3.2,unknown,162779:173998,,,,0,0,0,0.0,0,0
4,20298_3_36,kmgs,c00004_gnlBGI..,region 4.1,NRPS;unknown,38144:91874,Equibactin,nrps,BGC0000347,1,16,4,10112.0,15,3
5,20298_3_36,kmgs,c00004_gnlBGI..,region 4.2,lanthipeptide,106401:129154,,,,0,0,0,0.0,0,0


In [331]:
c = get_antismash_and_knowncluster("results/kmgs/1348662.3.patric/1348662.3.patric.json")
c

,bin_id,mgs_type,record_id,record_region,protocluster_type,bgc_location,kc_most_similar,kc_known_cluster,kc_bgc_acc,kc_total_hits,kc_hits,kc_core_gene_hits,kc_blast_score,kc_synteny_score,kc_core_bonus
0,1348662.3.patric,kmgs,c00001_gnlBGI..,region 1.1,unknown,57555:72801,,,,0,0,0,0.0,0,0
1,1348662.3.patric,kmgs,c00001_gnlBGI..,region 1.2,unknown,135989:149404,Kosinostatin,nrps-t2pks,BGC0001073,37,2,0,766.0,1,0
2,1348662.3.patric,kmgs,c00001_gnlBGI..,region 1.3,unknown,260616:268888,,,,0,0,0,0.0,0,0
3,1348662.3.patric,kmgs,c00001_gnlBGI..,region 1.4,betalactone;unknown,621495:648500,,,,0,0,0,0.0,0,0
4,1348662.3.patric,kmgs,c00001_gnlBGI..,region 1.5,terpene,1005578:1025704,SF2575,t2pks-saccharide-other,BGC0000269,3,3,0,656.0,2,0
5,1348662.3.patric,kmgs,c00001_gnlBGI..,region 1.6,unknown,1387922:1404500,,,,0,0,0,0.0,0,0
6,1348662.3.patric,kmgs,c00001_gnlBGI..,region 1.7,terpene;unknown;NRPS;T1PKS;unknown,1488186:1556538,Carotenoid,terpene,BGC0000636,2,2,1,598.0,0,3
7,1348662.3.patric,kmgs,c00001_gnlBGI..,region 1.8,unknown,1741036:1750337,,,,0,0,0,0.0,0,0
8,1348662.3.patric,kmgs,c00001_gnlBGI..,region 1.9,T1PKS;unknown,1902144:1948912,,,,0,0,0,0.0,0,0


In [333]:
## extract data ##
json_files = []
with open("antismash_kmgs_json.list", 'r') as ih:
    for line in ih:
        json_files.append(line.strip())
with open("antismash_umgs_json.list", "r") as ih2:
    for line in ih2:
        json_files.append(line.strip())
        
df_ptotocluster = get_all_antismash(json_files, get_antismash_protocluster, 8)
df_ptotocluster.to_csv("all_bgc_protocluster.tsv", sep='\t', index=False)

df_kc = get_all_antismash(json_files, get_antismash_and_knowncluster, 8)
df_kc.to_csv("all_bgc_and_knowncluster.tsv", sep='\t', index=False)

error occurred while parsing json: [Errno 2] No such file or directory: 'results/kmgs/SEQF1079/SEQF1079.json'


In [172]:
## analysis ##

In [225]:
metadata = pd.read_csv("all_fna_v2_and_metainfo.tsv", sep='\t', low_memory=False)
def set_bin_id(row):
    if row["fna_path"].endswith(".gz"):
        return os.path.basename(row["fna_path"]).rpartition(".")[0].rpartition(".")[0]
    else:
        return os.path.basename(row["fna_path"]).rpartition(".")[0]

metadata["bin_id"] =  metadata.apply(lambda x: set_bin_id(x), axis=1)

metadata.head()

,fna_path,id,project,accession,country,city,oral_site,age,sex,disease,assembly_or_ref,id_2,group,bin_id
0,/hwfssz1/ST_META/P18Z10200N0127_MA/zhujie/pub_...,ERR2764996.14,Georg-2014,ERR2764996,France,France,saliva,76.0,F,Small adenoma,assembly,ERR2764996.metaspades.bin.14,France,ERR2764996.metaspades.bin.14
1,/hwfssz1/ST_META/P18Z10200N0127_MA/zhujie/pub_...,SRR6748133.8,GR-2018,SRR6748133,USA,other,saliva,38.0,F,"Type 2 Diabetes, obesity",assembly,SRR6748133.metaspades.bin.8,USA,SRR6748133.metaspades.bin.8
2,/hwfssz1/ST_META/P18Z10200N0127_MA/zhujie/pub_...,SRR6748185.1,GR-2018,SRR6748185,USA,Caucasian,saliva,29.0,F,NaN,assembly,SRR6748185.metaspades.bin.1,USA,SRR6748185.metaspades.bin.1
3,/hwfssz1/ST_META/P18Z10200N0127_MA/zhujie/pub_...,SRR8114071.7,Heintz-2016,SRR8114071,Luxembourg,Na,saliva,16.0,F,NaN,assembly,SRR8114071.metaspades.bin.7,Luxembourg,SRR8114071.metaspades.bin.7
4,/hwfssz1/ST_META/P18Z10200N0127_MA/zhujie/pub_...,ERR2764930.9,Georg-2014,ERR2764930,France,France,saliva,55.0,F,Cancer,assembly,ERR2764930.metaspades.bin.9,France,ERR2764930.metaspades.bin.9


In [226]:
taxonomy = pd.read_csv("oral_mgs_gtdb_taxonomy_add_old_tax.tsv", sep='\t')
taxonomy.head()

,mgs_id,size,mtype,representative,MAG,oral_genome,rep_path,genome_lst,mgs_id_old,bin_id,...,classification_new,lineages_superkingdom_new,lineages_phylum_new,lineages_class_new,lineages_order_new,lineages_family_new,lineages_genus_new,lineages_species_new,lineages_strain_new,lineages_old
0,mgs_1,1289,kMGS,GCF_002013135.1_ASM201313v1,0,12,/hwfssz1/pub/database/ftp.ncbi.nih.gov/genomes...,"GCF_000349405.1_ASM34940v1,GCF_004013755.1_ASM...",mgs_d0.05_1,GCF_002013135.1_ASM201313v1_genomic,...,d__Bacteria;p__Campylobacterota;c__Campylobact...,k__Bacteria,k__Bacteria|p__Campylobacterota,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k_Bacteria|p_Proteobacteria|c_Epsilonproteobac...
1,mgs_2,1576,kMGS,YS000214_saliva.17,1572,1574,/hwfssz1/ST_META/P18Z10200N0127_MA/zhujie/yunn...,"RSZAXPI002486-19_RAH_saliva.49,RDPYD18300072_A...",mgs_d0.05_5,YS000214_saliva.spades.bin.17,...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,k__Bacteria,k__Bacteria|p__Bacteroidota,k__Bacteria|p__Bacteroidota|c__Bacteroidia,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k_Bacteria|p_Bacteroidetes|c_Bacteroidia|o_Bac...
2,mgs_3,6,kMGS,1150461.5.patric.fna,0,2,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,"SEQF2707,1150461.5.patric.fna,GCF_001005065.1_...",mgs_d0.05_9,1150461.5.patric,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
3,mgs_4,8301,kMGS,GCF_900050145.1_6938_4_11,0,52,/hwfssz1/pub/database/ftp.ncbi.nih.gov/genomes...,"GCF_001141485.1_6925_1_59,GCF_900022795.1_1186...",mgs_d0.05_13,GCF_900050145.1_6938_4_11_genomic,...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k_Bacteria|p_Firmicutes|c_Bacilli|o_Lactobacil...
4,mgs_5,1590,kMGS,GCF_001544055.1_ASM154405v1,0,1,/hwfssz1/pub/database/ftp.ncbi.nih.gov/genomes...,"GCF_000391785.1_Ente_faec_7230532-1_V1,GCF_002...",mgs_d0.05_14,GCF_001544055.1_ASM154405v1_genomic,...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k_Bacteria|p_Firmicutes|c_Bacilli|o_Lactobacil...


In [334]:
df_protocluster = pd.read_csv("all_bgc_protocluster.tsv", sep='\t', keep_default_na=False)
df_kc = pd.read_csv("all_bgc_and_knowncluster.tsv", sep='\t', keep_default_na=False)

In [335]:
df_protocluster.head()

,bin_id,mgs_type,record_id,bgc_location,bgc_core_location,detection_rule,protocluster_type,gene_list
0,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00002_gnlBGI..,[192968:201394],,,unknown,
1,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,[147358:157694],[152358:152694](+),(strepbact or Antimicrobial14 or Bacteriocin_I...,bacteriocin,AFDDOBBN_01092
2,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,[155792:171350],,,unknown,
3,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00006_gnlBGI..,[45795:55219],,,unknown,
4,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00011_gnlBGI..,[24502:67778],[44502:47778](-),cds((PP-binding or NAD_binding_4) and (AMP-bin...,NRPS-like,AFDDOBBN_01813;AFDDOBBN_01821


In [229]:
df_kc.head(11)

,bin_id,mgs_type,record_id,record_region,protocluster_type,bgc_location,kc_most_similar,kc_known_cluster,kc_bgc_acc,kc_total_hits,kc_hits,kc_core_gene_hits,kc_blast_score,kc_synteny_score,kc_core_bonus
0,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00002_gnlBGI..,region 2.1,unknown,192969:201394,Chlorizidine A,nrps-t1pks,BGC0001172,1,2,0,471.0,1,0
1,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,region 4.1,bacteriocin;unknown,147359:171350,,,,0,0,0,0.0,0,0
2,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00006_gnlBGI..,region 6.1,unknown,45796:55219,Heme D1,other,BGC0000906,2,2,0,642.0,0,0
3,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00011_gnlBGI..,region 11.1,NRPS-like;unknown,24503:67778,Phenazine,other,BGC0001080,4,2,0,518.0,1,0
4,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00013_gnlBGI..,region 13.1,NRPS;unknown,6266:66724,Kanamycin,saccharide,BGC0000703,14,2,0,949.0,0,0
5,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00014_gnlBGI..,region 14.1,unknown,54719:61208,Polysaccharide B,saccharide,BGC0001411,1,2,0,698.0,1,0
6,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00015_gnlBGI..,region 15.1,unknown,31345:49283,,,,0,0,0,0.0,0,0
7,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00018_gnlBGI..,region 18.1,bacteriocin;unknown,23771:41001,Phosphonoglycans,saccharide,BGC0000806,1,4,0,1498.0,2,0
8,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00022_gnlBGI..,region 22.1,unknown,26540:36368,Kosinostatin,nrps-t2pks,BGC0001073,57,3,0,928.0,1,0
9,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00024_gnlBGI..,region 24.1,lanthipeptide,1524:24064,,,,0,0,0,0.0,0,0


In [230]:
df_kc_info = df_kc.merge(metadata).merge(taxonomy)

In [336]:
df_protocluster_info = df_protocluster.merge(metadata).merge(taxonomy)

In [360]:
df_protocluster_info.to_csv("df_protocluster_info.tsv", sep='\t', index=False)

In [337]:
[len(df_kc), len(df_kc_info)]

[11391, 11391]

In [338]:
[len(df_protocluster), len(df_protocluster_info)]

[12399, 12399]

In [232]:
df_kc_info.head()

,bin_id,mgs_type,record_id,record_region,protocluster_type,bgc_location,kc_most_similar,kc_known_cluster,kc_bgc_acc,kc_total_hits,...,classification_new,lineages_superkingdom_new,lineages_phylum_new,lineages_class_new,lineages_order_new,lineages_family_new,lineages_genus_new,lineages_species_new,lineages_strain_new,lineages_old
0,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00002_gnlBGI..,region 2.1,unknown,192969:201394,Chlorizidine A,nrps-t1pks,BGC0001172,1,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
1,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,region 4.1,bacteriocin;unknown,147359:171350,,,,0,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
2,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00006_gnlBGI..,region 6.1,unknown,45796:55219,Heme D1,other,BGC0000906,2,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
3,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00011_gnlBGI..,region 11.1,NRPS-like;unknown,24503:67778,Phenazine,other,BGC0001080,4,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
4,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00013_gnlBGI..,region 13.1,NRPS;unknown,6266:66724,Kanamycin,saccharide,BGC0000703,14,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...


In [373]:
df_protocluster_info.head()

,bin_id,mgs_type,record_id,bgc_location,bgc_core_location,detection_rule,protocluster_type,gene_list,fna_path,id,...,classification_new,lineages_superkingdom_new,lineages_phylum_new,lineages_class_new,lineages_order_new,lineages_family_new,lineages_genus_new,lineages_species_new,lineages_strain_new,lineages_old
0,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00002_gnlBGI..,[192968:201394],,,unknown,,/hwfssz1/ST_META/F16ZQSB1SY2794_RO_GRJ/RO2/res...,RSZAXPI002328-45_RAH_dental.69,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
1,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,[147358:157694],[152358:152694](+),(strepbact or Antimicrobial14 or Bacteriocin_I...,bacteriocin,AFDDOBBN_01092,/hwfssz1/ST_META/F16ZQSB1SY2794_RO_GRJ/RO2/res...,RSZAXPI002328-45_RAH_dental.69,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
2,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,[155792:171350],,,unknown,,/hwfssz1/ST_META/F16ZQSB1SY2794_RO_GRJ/RO2/res...,RSZAXPI002328-45_RAH_dental.69,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
3,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00006_gnlBGI..,[45795:55219],,,unknown,,/hwfssz1/ST_META/F16ZQSB1SY2794_RO_GRJ/RO2/res...,RSZAXPI002328-45_RAH_dental.69,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
4,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00011_gnlBGI..,[24502:67778],[44502:47778](-),cds((PP-binding or NAD_binding_4) and (AMP-bin...,NRPS-like,AFDDOBBN_01813;AFDDOBBN_01821,/hwfssz1/ST_META/F16ZQSB1SY2794_RO_GRJ/RO2/res...,RSZAXPI002328-45_RAH_dental.69,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...


In [ ]:
## 1

In [280]:
def set_protocluster_type(row):
    if row["protocluster_type"] == "unknown":
        return "unknown"
    else:
        for i in row["protocluster_type"].split(";"):
            if i != "unknown":
                return "known"
        return "unknown"

df_kc_info["protocluster_type2"] = df_kc_info.apply(lambda x: set_protocluster_type(x), axis=1)
df_kc_info.to_csv("df_kc_info.tsv", sep='\t', index=False)

In [434]:
df_protocluster_count_known = df_protocluster_info.query('protocluster_type != "unknown" and protocluster_type != "other"')\
                                                  .groupby(["mgs_type", "protocluster_type"]).size().to_frame().rename(columns={0: "bgc_count"}).reset_index()

df_protocluster_count_known2 = df_protocluster_info.groupby(["mgs_type", "protocluster_type"]).size().to_frame().rename(columns={0: "bgc_count"}).reset_index()

In [435]:
df_protocluster_count_total = df_protocluster_count_known.groupby("protocluster_type").sum().reset_index().rename(columns={"bgc_count": "bgc_count_total"})
df_protocluster_count_known = df_protocluster_count_known.merge(df_protocluster_count_total).sort_values("bgc_count_total", ascending=False)

df_protocluster_count_total2 = df_protocluster_count_known2.groupby("protocluster_type").sum().reset_index().rename(columns={"bgc_count": "bgc_count_total"})
df_protocluster_count_known2 = df_protocluster_count_known2.merge(df_protocluster_count_total2).sort_values("bgc_count_total", ascending=False)

In [437]:
df_protocluster_count_known.to_csv("df_protocluster_count_known.tsv", sep='\t', index=False)
df_protocluster_count_known2.to_csv("df_protocluster_count_known2.tsv", sep='\t', index=False)

In [283]:
df_kc_count = df_kc_info.groupby(["mgs_type", "kc_known_cluster"]).size().to_frame().rename(columns={0: "bgc_count"}).reset_index()
df_kc_count2 = df_kc_info.query('protocluster_type2 != "unknown"').groupby(["mgs_type", "kc_known_cluster"]).size().to_frame().rename(columns={0: "bgc_count"}).reset_index()

In [284]:
df_kc_count_known = df_kc_count.query('kc_known_cluster != "" and kc_known_cluster != "other"')
df_kc_count_known2 = df_kc_count2.query('kc_known_cluster != "" and kc_known_cluster != "other"')

In [285]:
['' in df_kc_count_known.kc_known_cluster.to_list(), 'other' in df_kc_count_known.kc_known_cluster.to_list()]

[False, False]

In [286]:
['' in df_kc_count_known2.kc_known_cluster.to_list(), 'other' in df_kc_count_known2.kc_known_cluster.to_list()]

[False, False]

In [287]:
df_kc_count_known_total = df_kc_count_known.groupby(["kc_known_cluster"]).sum().reset_index().rename(columns={"bgc_count" : "bgc_count_total"})
df_kc_count_known_total2 = df_kc_count_known2.groupby(["kc_known_cluster"]).sum().reset_index().rename(columns={"bgc_count" : "bgc_count_total"})

In [288]:
df_kc_count_known = df_kc_count_known.merge(df_kc_count_known_total).sort_values("bgc_count_total", ascending=False)
df_kc_count_known2 = df_kc_count_known2.merge(df_kc_count_known_total2).sort_values("bgc_count_total", ascending=False)

In [294]:
df_kc_count_known.to_csv("df_kc_count_known.tsv", sep='\t', index=False)
df_kc_count_known2.to_csv("df_kc_count_known2.tsv", sep='\t', index=False)

In [291]:
df_kc_count_known

,mgs_type,kc_known_cluster,bgc_count,bgc_count_total
24,umgs,saccharide,431,1517
23,kmgs,saccharide,1086,1517
35,kmgs,t2pks,189,360
36,umgs,t2pks,171,360
7,kmgs,nrps,291,319
8,umgs,nrps,28,319
28,kmgs,t1pks,106,141
29,umgs,t1pks,35,141
19,kmgs,polyketide,80,128
20,umgs,polyketide,48,128


In [292]:
df_kc_count_known2

,mgs_type,kc_known_cluster,bgc_count,bgc_count_total
7,kmgs,nrps,210,235
8,umgs,nrps,25,235
17,kmgs,polyketide,57,98
18,umgs,polyketide,41,98
3,kmgs,lanthipeptide,43,51
4,umgs,lanthipeptide,8,51
22,umgs,saccharide,3,46
21,kmgs,saccharide,43,46
37,umgs,terpene,1,43
36,kmgs,terpene,42,43


In [279]:
##

In [364]:
df_protocluster_info.head()

,bin_id,mgs_type,record_id,bgc_location,bgc_core_location,detection_rule,protocluster_type,gene_list,fna_path,id,...,classification_new,lineages_superkingdom_new,lineages_phylum_new,lineages_class_new,lineages_order_new,lineages_family_new,lineages_genus_new,lineages_species_new,lineages_strain_new,lineages_old
0,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00002_gnlBGI..,[192968:201394],,,unknown,,/hwfssz1/ST_META/F16ZQSB1SY2794_RO_GRJ/RO2/res...,RSZAXPI002328-45_RAH_dental.69,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
1,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,[147358:157694],[152358:152694](+),(strepbact or Antimicrobial14 or Bacteriocin_I...,bacteriocin,AFDDOBBN_01092,/hwfssz1/ST_META/F16ZQSB1SY2794_RO_GRJ/RO2/res...,RSZAXPI002328-45_RAH_dental.69,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
2,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,[155792:171350],,,unknown,,/hwfssz1/ST_META/F16ZQSB1SY2794_RO_GRJ/RO2/res...,RSZAXPI002328-45_RAH_dental.69,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
3,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00006_gnlBGI..,[45795:55219],,,unknown,,/hwfssz1/ST_META/F16ZQSB1SY2794_RO_GRJ/RO2/res...,RSZAXPI002328-45_RAH_dental.69,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...
4,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00011_gnlBGI..,[24502:67778],[44502:47778](-),cds((PP-binding or NAD_binding_4) and (AMP-bin...,NRPS-like,AFDDOBBN_01813;AFDDOBBN_01821,/hwfssz1/ST_META/F16ZQSB1SY2794_RO_GRJ/RO2/res...,RSZAXPI002328-45_RAH_dental.69,...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...


In [365]:
df_protocluster_count_by_bin = df_protocluster_info.groupby(["bin_id", "protocluster_type"]).size().to_frame().rename(columns={0: "count"})

In [366]:
df_protocluster_count_by_bin.head()

count
bin_id           protocluster_type       
1000588.3.patric T3PKS                  1
                 bacteriocin            2
                 unknown                4
1008453.3.patric T3PKS                  1
                 bacteriocin            1

In [367]:
df_protocluster_count_info_by_bin = df_protocluster_count_by_bin.reset_index().merge(metadata).merge(taxonomy)

In [368]:
df_protocluster_count_info_by_bin.head()

,bin_id,protocluster_type,count,fna_path,id,project,accession,country,city,oral_site,...,classification_new,lineages_superkingdom_new,lineages_phylum_new,lineages_class_new,lineages_order_new,lineages_family_new,lineages_genus_new,lineages_species_new,lineages_strain_new,lineages_old
0,1000588.3.patric,T3PKS,1,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,1000588.3.patric.fna,NaN,NaN,NaN,NaN,NaN,...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k_Bacteria|p_Firmicutes|c_Bacilli|o_Lactobacil...
1,1000588.3.patric,bacteriocin,2,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,1000588.3.patric.fna,NaN,NaN,NaN,NaN,NaN,...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k_Bacteria|p_Firmicutes|c_Bacilli|o_Lactobacil...
2,1000588.3.patric,unknown,4,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,1000588.3.patric.fna,NaN,NaN,NaN,NaN,NaN,...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k_Bacteria|p_Firmicutes|c_Bacilli|o_Lactobacil...
3,1008453.3.patric,T3PKS,1,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,1008453.3.patric.fna,NaN,NaN,NaN,NaN,NaN,...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k_Bacteria|p_Firmicutes|c_Bacilli|o_Lactobacil...
4,1008453.3.patric,bacteriocin,1,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,1008453.3.patric.fna,NaN,NaN,NaN,NaN,NaN,...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k_Bacteria|p_Firmicutes|c_Bacilli|o_Lactobacil...


In [369]:
df_protocluster_count_info_by_bin.to_csv("df_protocluster_count_info_by_bin.tsv", sep='\t', index=False)

In [370]:
[len(df_protocluster_count_by_bin), len(df_protocluster_count_info_by_bin)]

[5844, 5844]

In [371]:
df_protocluster_count_mat_by_bin = df_protocluster_count_by_bin.unstack('protocluster_type', fill_value=0)

In [372]:
df_protocluster_count_mat_by_bin.head()

count                                                   \
protocluster_type  CDPS LAP NAGGN NRPS NRPS-like PKS-like RaS-RiPP T1PKS   
bin_id                                                                     
1000588.3.patric      0   0     0    0         0        0        0     0   
1008453.3.patric      0   0     0    0         0        0        0     0   
1009846.3.patric      0   0     0    1         0        0        0     1   
1017.4.patric         0   0     0    0         0        0        0     0   
1073353.3.patric      0   0     0    0         0        0        0     0   

                               ...                                   \
protocluster_type T2PKS T3PKS  ... phosphonate proteusin resorcinol   
bin_id                         ...                                    
1000588.3.patric      0     1  ...           0         0          0   
1008453.3.patric      0     1  ...           0         0          0   
1009846.3.patric      0     0  ...           1         0          0   
1017.4.patric         0     0  ...           0         0          1   
1073353.3.patric      0     0  ...           0         0          0   

                                                                            \
protocluster_type sactipeptide siderophore terpene thiopeptide transAT-PKS   
bin_id                                                                       
1000588.3.patric             0           0       0           0           0   
1008453.3.patric             0           0       0           0           0   
1009846.3.patric             0           0       3           0           0   
1017.4.patric                0           0       0           0           0   
1073353.3.patric             0           0       0           0           0   

                                            
protocluster_type transAT-PKS-like unknown  
bin_id                                      
1000588.3.patric                 0       4  
1008453.3.patric                 0       3  
1009846.3.patric                 0      25  
1017.4.patric                    0       5  
1073353.3.patric                 0       8  

[5 rows x 40 columns]

In [383]:
##

In [384]:
df_kc_info.head()

,bin_id,mgs_type,record_id,record_region,protocluster_type,bgc_location,kc_most_similar,kc_known_cluster,kc_bgc_acc,kc_total_hits,...,lineages_superkingdom_new,lineages_phylum_new,lineages_class_new,lineages_order_new,lineages_family_new,lineages_genus_new,lineages_species_new,lineages_strain_new,lineages_old,protocluster_type2
0,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00002_gnlBGI..,region 2.1,unknown,192969:201394,Chlorizidine A,nrps-t1pks,BGC0001172,1,...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...,unknown
1,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,region 4.1,bacteriocin;unknown,147359:171350,,,,0,...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...,known
2,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00006_gnlBGI..,region 6.1,unknown,45796:55219,Heme D1,other,BGC0000906,2,...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...,unknown
3,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00011_gnlBGI..,region 11.1,NRPS-like;unknown,24503:67778,Phenazine,other,BGC0001080,4,...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...,known
4,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00013_gnlBGI..,region 13.1,NRPS;unknown,6266:66724,Kanamycin,saccharide,BGC0000703,14,...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...,known


In [385]:
mgs_stats = pd.read_csv("stats.tsv", sep='\t')
mgs_stats = mgs_stats.assign(bin_id=mgs_stats.file.apply(lambda x: x.rpartition(".")[0]))
mgs_stats.head()

,file,format,type,num_seqs,sum_len,min_len,avg_len,max_len,bin_id
0,1000588.3.patric.fa,FASTA,DNA,66,2020730,520,30617.1,203739,1000588.3.patric
1,1008453.3.patric.fa,FASTA,DNA,34,1943752,596,57169.2,318811,1008453.3.patric
2,1009846.3.patric.fa,FASTA,DNA,2,6467321,3003666,3233660.5,3463655,1009846.3.patric
3,1017.4.patric.fa,FASTA,DNA,1,2838633,2838633,2838633.0,2838633,1017.4.patric
4,1073353.3.patric.fa,FASTA,DNA,274,2192767,205,8002.8,65929,1073353.3.patric


In [388]:
df_kc_info = df_kc_info.merge(mgs_stats)

In [390]:
df_kc_info.head()

,bin_id,mgs_type,record_id,record_region,protocluster_type,bgc_location,kc_most_similar,kc_known_cluster,kc_bgc_acc,kc_total_hits,...,lineages_old,protocluster_type2,file,format,type,num_seqs,sum_len,min_len,avg_len,max_len
0,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00002_gnlBGI..,region 2.1,unknown,192969:201394,Chlorizidine A,nrps-t1pks,BGC0001172,1,...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...,unknown,RSZAXPI002328-45_RAH_dental.metaspades.bin.69.fa,FASTA,DNA,30,2728871,4253,90962.4,402931
1,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,region 4.1,bacteriocin;unknown,147359:171350,,,,0,...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...,known,RSZAXPI002328-45_RAH_dental.metaspades.bin.69.fa,FASTA,DNA,30,2728871,4253,90962.4,402931
2,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00006_gnlBGI..,region 6.1,unknown,45796:55219,Heme D1,other,BGC0000906,2,...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...,unknown,RSZAXPI002328-45_RAH_dental.metaspades.bin.69.fa,FASTA,DNA,30,2728871,4253,90962.4,402931
3,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00011_gnlBGI..,region 11.1,NRPS-like;unknown,24503:67778,Phenazine,other,BGC0001080,4,...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...,known,RSZAXPI002328-45_RAH_dental.metaspades.bin.69.fa,FASTA,DNA,30,2728871,4253,90962.4,402931
4,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00013_gnlBGI..,region 13.1,NRPS;unknown,6266:66724,Kanamycin,saccharide,BGC0000703,14,...,k_Bacteria|p_Actinobacteria|c_Actinobacteria|o...,known,RSZAXPI002328-45_RAH_dental.metaspades.bin.69.fa,FASTA,DNA,30,2728871,4253,90962.4,402931


In [409]:
def bgc_rate(row):
    begin, end = row["bgc_location"].split(":")
    return (int(end) - int(begin) + 1) / row["sum_len"] * 100

In [410]:
df_kc_info["bgc_rate"] = df_kc_info.apply(lambda x: bgc_rate(x), axis=1)

In [411]:
df_kc_info.head()

,bin_id,mgs_type,record_id,record_region,protocluster_type,bgc_location,kc_most_similar,kc_known_cluster,kc_bgc_acc,kc_total_hits,...,protocluster_type2,file,format,type,num_seqs,sum_len,min_len,avg_len,max_len,bgc_rate
0,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00002_gnlBGI..,region 2.1,unknown,192969:201394,Chlorizidine A,nrps-t1pks,BGC0001172,1,...,unknown,RSZAXPI002328-45_RAH_dental.metaspades.bin.69.fa,FASTA,DNA,30,2728871,4253,90962.4,402931,0.308772
1,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00004_gnlBGI..,region 4.1,bacteriocin;unknown,147359:171350,,,,0,...,known,RSZAXPI002328-45_RAH_dental.metaspades.bin.69.fa,FASTA,DNA,30,2728871,4253,90962.4,402931,0.879191
2,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00006_gnlBGI..,region 6.1,unknown,45796:55219,Heme D1,other,BGC0000906,2,...,unknown,RSZAXPI002328-45_RAH_dental.metaspades.bin.69.fa,FASTA,DNA,30,2728871,4253,90962.4,402931,0.345344
3,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00011_gnlBGI..,region 11.1,NRPS-like;unknown,24503:67778,Phenazine,other,BGC0001080,4,...,known,RSZAXPI002328-45_RAH_dental.metaspades.bin.69.fa,FASTA,DNA,30,2728871,4253,90962.4,402931,1.585857
4,RSZAXPI002328-45_RAH_dental.metaspades.bin.69,kmgs,c00013_gnlBGI..,region 13.1,NRPS;unknown,6266:66724,Kanamycin,saccharide,BGC0000703,14,...,known,RSZAXPI002328-45_RAH_dental.metaspades.bin.69.fa,FASTA,DNA,30,2728871,4253,90962.4,402931,2.215532


In [427]:
df_kc_info_rate = df_kc_info.groupby("bin_id")["bgc_rate"].sum().to_frame().reset_index()

In [428]:
df_kc_info_rate.head()

,bin_id,bgc_rate
0,1000588.3.patric,4.400241
1,1008453.3.patric,4.313873
2,1009846.3.patric,9.887355
3,1017.4.patric,9.143627
4,1073353.3.patric,2.610355


In [429]:
df_kc_info_rate = df_kc_info_rate.merge(metadata).merge(taxonomy)

In [430]:
len(df_kc_info_rate)

2713

In [431]:
df_kc_info_rate.head()

,bin_id,bgc_rate,fna_path,id,project,accession,country,city,oral_site,age,...,classification_new,lineages_superkingdom_new,lineages_phylum_new,lineages_class_new,lineages_order_new,lineages_family_new,lineages_genus_new,lineages_species_new,lineages_strain_new,lineages_old
0,1000588.3.patric,4.400241,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,1000588.3.patric.fna,NaN,NaN,NaN,NaN,NaN,NaN,...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k_Bacteria|p_Firmicutes|c_Bacilli|o_Lactobacil...
1,1008453.3.patric,4.313873,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,1008453.3.patric.fna,NaN,NaN,NaN,NaN,NaN,NaN,...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k_Bacteria|p_Firmicutes|c_Bacilli|o_Lactobacil...
2,1009846.3.patric,9.887355,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,1009846.3.patric.fna,NaN,NaN,NaN,NaN,NaN,NaN,...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,k__Bacteria,k__Bacteria|p__Proteobacteria,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,k__Bacteria|p__Proteobacteria|c__Gammaproteoba...,k_Bacteria|p_Proteobacteria|c_Gammaproteobacte...
3,1017.4.patric,9.143627,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,1017.4.patric.fna,NaN,NaN,NaN,NaN,NaN,NaN,...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,k__Bacteria,k__Bacteria|p__Bacteroidota,k__Bacteria|p__Bacteroidota|c__Bacteroidia,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k_Bacteria|p_Bacteroidetes|c_Bacteroidia|o_Fla...
4,1073353.3.patric,2.610355,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,1073353.3.patric.fna,NaN,NaN,NaN,NaN,NaN,NaN,...,d__Bacteria;p__Campylobacterota;c__Campylobact...,k__Bacteria,k__Bacteria|p__Campylobacterota,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k_Bacteria|p_Epsilonbacteraeota|c_Campylobacte...


In [421]:
df_kc_info_rate.to_csv("df_kc_info_rate.tsv", sep='\t', index=False)

In [433]:
np.mean(df_kc_info_rate["bgc_rate"])

2.5121236149515584